# 快速启动

## 构建一个基本的 Agent

In [ ]:
from langchain.agents import create_agent


def get_weather(city: str) -> str:
  """Get weather for a given city."""
  return f"It's always sunny in {city}!"


agent = create_agent(
  model="gpt-3.5-turbo",
  tools=[get_weather],
  prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

## 构建一个现实世界的代理

In [ ]:
from langchain.agents import create_agent

# 1.定义系统提示
system_prompt = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean whereever they are, use the get_user_location tool to find their location."""

# 2.创建工具
from langchain_core.tools import tool

def get_weather_for_location(city: str) -> str:  # (1)!
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

from langchain_core.runnables import RunnableConfig

USER_LOCATION = {
    "1": "Florida",
    "2": "SF"
}

@tool
def get_user_location(config: RunnableConfig) -> str:
    """Retrieve user information based on user ID."""
    user_id = config["configurable"].get("user_id")
    return USER_LOCATION[user_id]


# 3.配置你的模型
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "openai:gpt-3.5-turbo",
    temperature=0
)

# 4.定义响应格式
from dataclasses import dataclass

@dataclass
class WeatherResponse:
    conditions: str
    punny_response: str
    
# 5.添加记忆
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

# 6.将所有内容整合在一起
agent = create_agent(
    model=model,
    prompt=system_prompt,
    tools=[get_user_location, get_weather_for_location],
    response_format=WeatherResponse,
    checkpointer=checkpointer
)

config = {
    "configurable": {
        "thread_id": "1",
        "user_id": "1"
    }
}
context = {"user_id": "1"}
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=context
)

response['structured_response']

response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=context
)

response['structured_response']